In [1]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
user_data = pd.read_csv('Data/user_data.csv')
ads_data = pd.read_csv('Data/ads_data.csv')
user_messages = pd.read_csv('Data/user_messages.csv')
user_messages_test = pd.read_csv('Data/user_messages_test.csv')
ads_data = ads_data.loc[ads_data.enabled==1]
ads_data.reset_index(drop=True, inplace=True)

In [3]:

# user_data.loc[user_data.event==1]['images_count']

user_data.head()
# ads_data.head()
# user_messages.head()
# user_messages_test.head()
# pd.unique(user_data.origin)
# user_data.shape
# len(pd.unique(ads_data.ad_id))
# user_data.loc[user_data.user_id==541]

ads_data.head()

,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0
2,2017-06-11 00:44:57,559,view,android,-34.688454,-58.333435,browse,1123948,4.0,899.0,22.0,1.0
3,2017-05-22 02:36:06,637,first_message,android,-34.548401,-58.485168,search,2273498,NaN,0.0,21.0,2.0
4,2017-06-08 15:45:35,509,view,android,-34.582088,-58.405041,home,2722493,3.0,1529.0,132.0,3.0


,ad_id,category_id,seller_id,creation_time,title,description,price,lat,long,source,enabled
0,3916,815,59,2015-12-16 09:09:25,Remera marca premium alemana,Casi nuevo y sin uso. Lo compre online y me qu...,150.0,NaN,NaN,android,1
1,11738,806,12298,2016-01-19 18:38:24,Juego de sillones estilo campo,Divino juego de sillones de diseño campo,1500.0,NaN,NaN,android,1
2,26552,800,2586,2016-02-27 18:53:19,Radio para tu auto wv con dos,Esta en perfecto estado lo saque de mi auto co...,1200.0,-34.603989,-58.458111,android,1
3,29177,811,4436,2016-03-01 19:47:02,Pantalla para Netbook,"Pantalla para netbook Acer, Aspire One modelo ...",800.0,-34.550674,-58.465057,android,1
4,30228,806,28747,2016-03-03 01:07:07,2 Estufas eskabe 5500 k,Estufa de tiro balanceado Eskabe 5500 k . Muy ...,1000.0,NaN,NaN,android,1


In [4]:
len(pd.unique(user_data.ad_id))
len(pd.unique(user_data.user_id))
np.sum(user_data.channel=='desktop')

535651

12365

52534

In [5]:
user_data.replace(to_replace={'event':{'view':0, 'first_message':1}}, inplace=True)
user_data.replace(to_replace={'origin':{'browse_search':0, 'search':0, 'browse':0, 'home':1,
                                       'notification_center':1, 'push':2, 'drawer':2, 'deeplink':1}}, inplace=True)
user_data.replace(to_replace={'channel': {'android': 0, 'ios':1, 'desktop':2}}, inplace=True )


In [6]:
user_data['event_time'] = pd.to_datetime(user_data.event_time)
user_data['event_month'] = user_data['event_time'].dt.month
user_data['event_day'] = user_data['event_time'].dt.day
user_data['hour'] = user_data['event_time'].dt.hour

In [7]:
ad_id = ads_data['ad_id']
dat = ads_data['category_id']
augmented_ads_data = pd.DataFrame(data=np.array(dat), index=ad_id)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [9]:
cols_norm = ['ad_messages', 'ad_views', 'ad_impressions']
n_cols_norm = ['n_' + x for x in cols_norm]
user_data[n_cols_norm] = user_data[cols_norm].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [10]:
ratio = [0.85, 0.2, 0.15]
i=0
temp = pd.Series(np.zeros([user_data.shape[0]]) )
for label in n_cols_norm:
    temp += ratio[i]*user_data[label]
    i+=1

user_data['composition']=temp

In [17]:
# user_data = user_data.sort_values(by=['origin', 'event_month', 'event_day', 'ad_messages', 'ad_views', 'ad_impressions'], ascending=[True, False, False, False, False, False])
# user_data = user_data.sort_values(by=['origin','composition'], ascending=[True, False])
user_data = user_data.sort_values(by=['event_month', 'event_day', 'ad_messages', 'ad_views', 'ad_impressions', 'images_count', 'event'], ascending=[ False, False, False, False, False, False, False])
# , 'ad_impressions', 'event'

In [18]:
a = augmented_ads_data.loc[np.array(user_data['ad_id'])] 

In [19]:
ads_for_cat={}
for c in pd.unique(user_messages_test.category_id):
    ad_id_for_c = a.loc[a[0]==c]     #  refer above cell for a
    ads_for_cat[c] = list(ad_id_for_c.index.drop_duplicates()[:10])
    

In [20]:
ads_id = pd.Series(np.array(np.arange(user_messages_test.shape[0])))
i=0
for u, c in user_messages_test.values:
    b = user_data.loc[user_data.user_id==u]
#     b = b.sort_values(by=['ad_messages', 'ad_views', 'ad_impressions'], ascending=[False, False, False])
    user_ads = pd.unique(b['ad_id'])
    
    category_ads = ads_data.loc[ads_data.category_id==c]
    cate_ads = pd.unique(category_ads['ad_id'])
    temp=[]
    j=0
    for ad in user_ads:
        if (ad in cate_ads and j<10):
            temp.append(ad)
            j+=1
    
    t0 = len(temp)
    if(t0<10):
        t1 = 10-t0
        t2 = ads_for_cat.get(c)[:t1]
        temp = temp + t2
#     print len(temp)
    ads_id[i] = str(temp)
    
    i+=1
    
    
    

In [21]:
submission = user_messages_test.copy()
submission['ads'] = ads_id

In [22]:
submission.to_csv('subm.csv', index=False)

## ----------------------------------------------------------------------------------------------------------------------------------